In [3]:
import pandas as pd
from tsnn.data_utils import *
from tsnn.models.lstnet import LSTNet
from tsnn.models.deepsense import DeepSense

In [15]:
batch_size = 64
sampling_step = 1
pred_delay = 24
pred_length = 1
nb_input_features = 321
input_cols = list(range(321))
timesteps_lstnet = 168
timesteps_deepsense = 1000
interest_vars = [315]

In [8]:
# Load data
raw_data = pd.read_csv('/Users/sofiene/Desktop/electricity.txt', header=None)

# Scale data
scaled_data, stats_df = scale_maxabs(raw_data)

# Split into inputs and targets
inputs, targets = inputs_targets_split(scaled_data, 
                                       input_cols=input_cols, 
                                       target_cols=interest_vars,
                                       samples_length=timesteps_lstnet, 
                                       pred_delay=pred_delay, 
                                       pred_length=pred_length)

# Train-Val-Test split
train_idx, val_idx, test_idx = train_val_split(targets, train_ratio=0.6, val_ratio=0.2)

In [10]:
# Create model
model = LSTNet(input_shape=(timesteps_lstnet, nb_input_features), 
               interest_vars=interest_vars)
model.compile(loss='mae', optimizer='adam')

In [11]:
# Prepare data generators
train_gen = sample_gen_rnn(inputs, targets, train_idx, timesteps_lstnet, sampling_step, batch_size)
val_gen = sample_gen_rnn(inputs, targets, val_idx, timesteps_lstnet, sampling_step, batch_size)
tr_steps_per_epoch = (train_idx[1] - train_idx[0] + 1) // batch_size + 1
val_steps_per_epoch = (val_idx[1] - val_idx[0] + 1) // batch_size + 1

# Train the model
model.fit_generator(generator=train_gen, 
                    steps_per_epoch=tr_steps_per_epoch, 
                    validation_data=val_gen, 
                    validation_steps=val_steps_per_epoch,
                    epochs=3, 
                    shuffle=False)

Epoch 1/3
245/245 [==============================] - 148s - loss: 0.1143 - val_loss: 0.0698

In [19]:
# Create model
deepsense = DeepSense(sensor_dims_list=[321], 
                      sequence_length=1000, 
                      time_window_tau=8, 
                      freq_domain=False, 
                      output_dim=1)
deepsense.compile(loss='mae', optimizer='adam')

In [20]:
# Prepare data generators
train_gen_ds = sample_gen_rnn(inputs, targets, train_idx, timesteps_deepsense, sampling_step, batch_size)
val_gen_ds = sample_gen_rnn(inputs, targets, val_idx, timesteps_deepsense, sampling_step, batch_size)
tr_steps_per_epoch_ds = (train_idx[1] - train_idx[0] + 1) // batch_size + 1
val_steps_per_epoch_ds = (val_idx[1] - val_idx[0] + 1) // batch_size + 1

# Train the model
deepsense.fit_generator(generator=train_gen_ds, 
                        steps_per_epoch=tr_steps_per_epoch_ds, 
                        validation_data=val_gen_ds,
                        validation_steps=val_steps_per_epoch_ds,
                        epochs=3, 
                        shuffle=False)

Epoch 1/3
  5/245 [..............................] - ETA: 2580s - loss: 0.1330

KeyboardInterrupt: 